## Step1
### Create a Dummy Data Generator in Python

In [0]:
import time
import random

def generate_random_numbers():
    while True:
        number = random.randint(50, 100)
        print(f"Generated Number: {number}")
        time.sleep(2)

if __name__ == "__main__":
    generate_random_numbers()

Generated Number: 98
Generated Number: 93
Generated Number: 72
Generated Number: 86
Generated Number: 60
Generated Number: 60
Generated Number: 70
Generated Number: 95
Generated Number: 51
Generated Number: 65
Generated Number: 81
Generated Number: 62
Generated Number: 91
Generated Number: 57
Generated Number: 69
Generated Number: 92
Generated Number: 66
Generated Number: 69
Generated Number: 58


## Step3
### Capture the Incoming Event Hub Stream Using Structured Streaming in Databricks

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("EventHubStreaming") \
    .getOrCreate()

# Define Event Hub connection parameters
event_hub_connection_str = "Endpoint=sb://demoeventhubnew.servicebus.windows.net/;SharedAccessKeyName=coedemo;SharedAccessKey=eGE8FbunCqJa6mrXw1/lARorurbgGgEFe+AEhN5cjqc=;EntityPath=my-event-hub"
event_hub_name = "my-event-hub"

# Define the Event Hub configurations
event_hub_config = {
    "eventhubs.connectionString": event_hub_connection_str,
    "eventhubs.consumerGroup": "$Default",
    "eventhubs.maxEventsPerTrigger": "1000"
}

# Read stream from Event Hub
df = spark.readStream \
    .format("eventhubs") \
    .options(**event_hub_config) \
    .load()

# Display the schema of the incoming data
df.printSchema()


root
 |-- body: binary (nullable = true)
 |-- partition: string (nullable = true)
 |-- offset: string (nullable = true)
 |-- sequenceNumber: long (nullable = true)
 |-- enqueuedTime: timestamp (nullable = true)
 |-- publisher: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- systemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



## Step4
### Add a "Risk" Column to the Stream

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import StringType, IntegerType

spark = SparkSession.builder.appName("EventHubStream").getOrCreate()

eventHubsConf = {
  'eventhubs.connectionString': event_hub_connection_str
}

# Read stream from Event Hub
df = spark.readStream \
  .format("eventhubs") \
  .options(**eventHubsConf) \
  .load()

# Convert the data and add the 'Risk' column
df = df.withColumn("value", df["body"].cast(StringType()).cast(IntegerType()))
df = df.withColumn("Risk", when(col("value") > 80, "High").otherwise("Low"))

# Write to another Event Hub or a destination like Azure Blob Storage
df.writeStream \
  .format("eventhubs") \
  .options(**event_hub_config) \
  .start()

## Step5
### Capture the Output Stream in a Separate Event Hub

In [0]:
# Define the connection string for the output Event Hub
output_event_hub_connection_str = "Endpoint=sb://<your-output-event-hub-namespace>.servicebus.windows.net/;SharedAccessKeyName=<your-policy-name>;SharedAccessKey=<your-access-key>;EntityPath=<your-output-event-hub-name>"

output_event_hub_config = {
    "eventhubs.connectionString": output_event_hub_connection_str,
    "eventhubs.partitionKey": "partitionKey",
    "eventhubs.maxEventsPerTrigger": "1000"
}

# Write the processed stream to the output Event Hub
query = processed_df.writeStream \
    .format("eventhubs") \
    .options(**output_event_hub_config) \
    .outputMode("append") \
    .start()

query.awaitTermination()


## Step6
### Connect the Output Stream to Power BI Using Stream Analytics
1.Create a Stream Analytics Job in Azure.
##### 
2.Set the input as your output Event Hub. 
#####  
3.Set the output as Power BI: - Authorize Power BI. - Create a new dataset and table.

## Step7
### Build a Real-time Dashboard in Power BI
1.Open Power BI and connect to the dataset created by your Stream Analytics job.
##### 
2.Create a new report and add a bar chart or a table to visualize the count of High and Low risk values.
##### 
3.Publish the report to your Power BI workspace.
##### 
4.Pin the visuals to a dashboard to create a real-time dashboard.
##### 
5.Write a query to count the number of High and Low risk values: